In [1]:
import numpy as np
import pandas as pd
import deepchem as dc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_max_pool
from torch_geometric.data import Batch, Data
from torch_geometric.loader import DataLoader
from sklearn import metrics


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading some Jax models, missing a dependency. No module named 'haiku'


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(30, 256)
        self.conv2 = GCNConv(256, 256)
        self.conv3 = GCNConv(256, 256)
        self.conv4 = GCNConv(256, 256)
        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(p=0.4)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        x = F.relu(x)
        x = global_max_pool(x, data.batch)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [3]:
def custom_collate(batch):
    data_list, target_list = zip(*batch)
    batch_data = Batch.from_data_list(data_list)
    batch_target = torch.stack(target_list)
    return batch_data, batch_target

In [4]:
def calculate_statistics(group):
    r2_train = group['r2_train']
    r2_test = group['r2_test']
    return pd.Series({
        'r2_train_mean': np.mean(r2_train),
        'r2_train_std': np.std(r2_train, ddof=0),
        'r2_test_mean': np.mean(r2_test),
        'r2_test_std': np.std(r2_test, ddof=0)
    })

In [5]:
torch.manual_seed(0)

scaler = StandardScaler()

epochs = 100
lr = 1e-2
wd = 3e-4

results = []
for random_state in range(10):
    torch.manual_seed(0)
    print(f'Random State: {random_state}')
    
    for d in ["kappa2", "peoevsa6", "bertzct", "ggi10", "atsc4i", "bcutp1l", "kappa3", "estatevsa3", "kier3", "aats8p", "kier2"]:
        torch.manual_seed(0)
        print('Source : ', d)
        dataset = d
        
        for c in ["Yield_CO"]:
            torch.manual_seed(0)
            df = pd.read_csv('data_Real/data_real.csv')
            smiles = df["SMILES"]
            featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
            X = featurizer.featurize(smiles)
            
            y = df[c]
            y = scaler.fit_transform(y.values.reshape(-1, 1)).flatten()
            
            data_train, data_test, target_train, target_test = train_test_split(X, y, test_size=0.5, random_state=random_state)
            target_train = torch.tensor(target_train, dtype=torch.float32)
            target_test = torch.tensor(target_test, dtype=torch.float32)

            data_train_list = []
            for graph_data in data_train:
                node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
                edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
                edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
                data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
                data_train_list.append(data)

            data_test_list = []
            for graph_data in data_test:
                node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
                edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
                edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
                data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
                data_test_list.append(data)

            train_loader = DataLoader(list(zip(data_train_list, target_train)), batch_size=len(data_train_list), collate_fn=custom_collate)
            test_loader = DataLoader(list(zip(data_test_list, target_test)), batch_size=len(data_test_list), collate_fn=custom_collate)

            model = Net()
            model.load_state_dict(torch.load(f'data_Random/model_{dataset}_sc.pth'))
            model.fc3 = nn.Linear(128, 1)
        
            model.train()
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
            criterion = nn.MSELoss()
        
            for param in model.conv1.parameters():
                param.requires_grad = False
            for param in model.conv2.parameters():
                param.requires_grad = False
            for param in model.conv3.parameters():
                param.requires_grad = False
            for param in model.conv4.parameters():
                param.requires_grad = False

            device = torch.device('cpu')
            model.to(device)

            for epoch in range(epochs):
                for data, target in train_loader:
                    data = data.to(device)
                    target = target.to(device)
                    optimizer.zero_grad()
                    out = model(data)
                    loss = criterion(out, target.view(-1, 1))
                    loss.backward()
                    optimizer.step()

            model.eval()
            pred_train = []
            for data, target in train_loader:
                data = data.to(device)
                with torch.no_grad():
                    out = model(data)
                pred_train.append(out.cpu().numpy())
            pred_train = np.concatenate(pred_train)

            pred_test = []
            for data, target in test_loader:
                data = data.to(device)
                with torch.no_grad():
                    out = model(data)
                pred_test.append(out.cpu().numpy())
            pred_test = np.concatenate(pred_test)

            pred_train = scaler.inverse_transform(pred_train)
            pred_test = scaler.inverse_transform(pred_test)
            target_train = scaler.inverse_transform(target_train.numpy().reshape(-1, 1)).flatten()
            target_test = scaler.inverse_transform(target_test.numpy().reshape(-1, 1)).flatten()

            r2_train_score = metrics.r2_score(target_train, pred_train)
            r2_test_score = metrics.r2_score(target_test, pred_test)

            results.append({'source': d, 'target': c, 'r2_train': r2_train_score, 'r2_test': r2_test_score})
            print(f'R2 test for {c} with random state {random_state}: {r2_test_score}')

results_df = pd.DataFrame(results)
gen_results = results_df.groupby(['source', 'target']).apply(calculate_statistics).reset_index()
gen_results

Random State: 0
Source :  kappa2
R2 test for Yield_CO with random state 0: 0.8016662001609802
Source :  peoevsa6
R2 test for Yield_CO with random state 0: 0.5987648367881775
Source :  bertzct
R2 test for Yield_CO with random state 0: 0.8099862337112427
Source :  ggi10
R2 test for Yield_CO with random state 0: 0.7328518629074097
Source :  atsc4i
R2 test for Yield_CO with random state 0: 0.6571838855743408
Source :  bcutp1l
R2 test for Yield_CO with random state 0: 0.6654859781265259
Source :  kappa3
R2 test for Yield_CO with random state 0: 0.7340476512908936
Source :  estatevsa3
R2 test for Yield_CO with random state 0: 0.7791979312896729
Source :  kier3
R2 test for Yield_CO with random state 0: 0.7135006189346313
Source :  aats8p
R2 test for Yield_CO with random state 0: 0.7731991410255432
Source :  kier2
R2 test for Yield_CO with random state 0: 0.8011120557785034
Random State: 1
Source :  kappa2
R2 test for Yield_CO with random state 1: 0.6143205761909485
Source :  peoevsa6
R2 test 

R2 test for Yield_CO with random state 9: 0.6222162246704102
Source :  bcutp1l
R2 test for Yield_CO with random state 9: 0.45049649477005005
Source :  kappa3
R2 test for Yield_CO with random state 9: 0.7288376688957214
Source :  estatevsa3
R2 test for Yield_CO with random state 9: 0.6648054122924805
Source :  kier3
R2 test for Yield_CO with random state 9: 0.6741446852684021
Source :  aats8p
R2 test for Yield_CO with random state 9: 0.6347373723983765
Source :  kier2
R2 test for Yield_CO with random state 9: 0.589625358581543


C:\Users\noton\AppData\Local\Temp\ipykernel_18160\3021910935.py:111: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gen_results = results_df.groupby(['source', 'target']).apply(calculate_statistics).reset_index()


,source,target,r2_train_mean,r2_train_std,r2_test_mean,r2_test_std
0,aats8p,Yield_CO,0.987845,0.007572,0.683793,0.110502
1,atsc4i,Yield_CO,0.977207,0.021776,0.624706,0.107381
2,bcutp1l,Yield_CO,0.749430,0.061869,0.493918,0.135667
3,bertzct,Yield_CO,0.982503,0.007939,0.776404,0.074193
4,estatevsa3,Yield_CO,0.990172,0.006359,0.713407,0.096395
5,ggi10,Yield_CO,0.982800,0.010105,0.691015,0.102657
6,kappa2,Yield_CO,0.969513,0.016456,0.723491,0.089512
7,kappa3,Yield_CO,0.982133,0.008386,0.750575,0.097414
8,kier2,Yield_CO,0.976782,0.018209,0.739846,0.083696
9,kier3,Yield_CO,0.977634,0.011671,0.737577,0.077057


In [6]:
gen_results.to_csv(f'result/Yield/onestep_{epochs}.csv')